In [1]:
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from datetime import datetime
import distutils.dir_util
from keras.callbacks import CSVLogger
from keras.utils.np_utils import to_categorical
import numpy as np

from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, Dropout
from keras.layers import ZeroPadding2D, MaxPooling2D, AveragePooling2D, Activation, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50


from keras import __version__
print(__version__)

Using TensorFlow backend.


2.0.3


In [2]:
# get data for visualisation later

from keras.preprocessing.image import ImageDataGenerator
def preprocess_input(x, data_format=None):
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 4D.
        data_format: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """

    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1.25)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#batch_size = 32

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/train',
        target_size=(224, 224),
        batch_size=28,
        #batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/valid',
        target_size=(224, 224),
        shuffle=False,
        #batch_size=batch_size,
        batch_size=20,
        class_mode='categorical')

Found 3276 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [3]:
RESULTS_DIR = '/home/ubuntu/data/fishing/results'

import bcolz
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="ResNet50_conv_feats/trn_"):
    filenames = load_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat').tolist()
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return filenames, conv_feats, labels

trn_filenames, trn_conv_features, trn_labels = load_precomputed_data("ResNet50_conv_feats/trn_")
val_filenames, val_conv_features, val_labels = load_precomputed_data("ResNet50_conv_feats/val_")

In [4]:

resnet_base = ResNet50(include_top=False, weights='imagenet')


In [5]:
#classifier_input_shape = resnet_base.layers[-1].output_shape[1:] 
# i.e. shape of conv features (produces (None, None, None, 2048))
classifier_input_shape = (1, 1, 2048)
# classifier_input_shape = resnet_base.layers[-1].output_shape[1:]
classifier_input = Input(shape=classifier_input_shape)

In [6]:
# Create classifier model

x= Flatten()(classifier_input)
x = Dense(8, activation='softmax')(x)
                                                     
classifier_model_v1 = Model(classifier_input, x)

#from keras.optimizers import SGD
classifier_model_v1.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
classifier_model_v1.fit(trn_conv_features, trn_labels,
                                          batch_size=32, 
                                          epochs=5,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True)

Train on 3276 samples, validate on 500 samples
Epoch 1/5
3276/3276 [==============================] - 0s - loss: 1.7746 - acc: 0.8544 - val_loss: 2.0673 - val_acc: 0.7900
Epoch 2/5
3276/3276 [==============================] - 0s - loss: 1.8020 - acc: 0.8513 - val_loss: 2.0924 - val_acc: 0.7860
Epoch 3/5
3276/3276 [==============================] - 0s - loss: 2.0581 - acc: 0.8104 - val_loss: 2.3132 - val_acc: 0.7720
Epoch 4/5
3276/3276 [==============================] - 0s - loss: 1.7886 - acc: 0.8553 - val_loss: 2.0169 - val_acc: 0.8240
Epoch 5/5
3276/3276 [==============================] - 0s - loss: 1.7374 - acc: 0.8724 - val_loss: 2.1442 - val_acc: 0.7920


In [9]:
# above - pulled in the resnet weights for the convolutional layers - , then added a very simple classifier to test 
# everything was working ok

# below - try to add a convolutional network and input the precomputed convolutional layers

In [10]:
nf=128; p=0.

In [11]:

# x = Flatten(input_shape=classifier_input_shape)(classifier_input)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
# BatchNormalization(axis=1)(x)
# MaxPooling2D()(x)
# Convolution2D(nf,3,3, activation='relu', border_mode='same')(x)
# BatchNormalization(axis=1)(x)
# MaxPooling2D()(x)
# Convolution2D(nf,3,3, activation='relu', border_mode='same')(x)
# BatchNormalization(axis=1)(x)
# MaxPooling2D((1,2))(x)
x = Conv2D(8,(3,3), padding='same')(x)
x = Dropout(p)(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax')(x)

classifier_model_v2 = Model(classifier_input, x)

classifier_model_v2.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
classifier_model_v2.output_shape

(None, 8)

In [14]:
classifier_model_v2.fit(trn_conv_features, trn_labels, 
                                          batch_size=32, 
                                          epochs=5,
                                          validation_data=(val_conv_features,val_labels),
                                          shuffle=True)

Train on 3276 samples, validate on 500 samples
Epoch 1/5
3276/3276 [==============================] - 1s - loss: 0.1627 - acc: 0.9533 - val_loss: 0.3434 - val_acc: 0.9040
Epoch 2/5
3276/3276 [==============================] - 1s - loss: 0.1280 - acc: 0.9625 - val_loss: 0.4054 - val_acc: 0.8720
Epoch 3/5
3276/3276 [==============================] - 1s - loss: 0.0960 - acc: 0.9753 - val_loss: 0.3157 - val_acc: 0.9060
Epoch 4/5
3276/3276 [==============================] - 1s - loss: 0.0700 - acc: 0.9829 - val_loss: 0.3350 - val_acc: 0.9100
Epoch 5/5
3276/3276 [==============================] - 1s - loss: 0.0526 - acc: 0.9902 - val_loss: 0.3886 - val_acc: 0.8840


In [15]:
classifier_model_v2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 1, 2048)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 128)         2359424   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 8)           9224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 8)           0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 8)                 0         
_________________________________________________________________
activation_50 (Activation)   (None, 8)                 0         
Total params: 2,368,648
Trainable params: 2,368,648
Non-trainable params: 0
_________________________________________________________________


In [18]:
l = classifier_model_v2.layers
input_layer = l[0]
last_conv_layer = l[-4]
conv_fn = K.function([input_layer.input], [last_conv_layer.output])

In [135]:
conv_fn = K.function([input_layer.input, K.learning_phase()], [last_conv_layer.output])
# output_image = output_fn([input_image])

In [45]:
l[-4]

In [71]:
import scipy
def get_cm(imp, label):
    conv = conv_fn([inp,0])[0,label]
#     print(conv.shape)
    return scipy.misc.imresize(conv, (360,640), interp='nearest')

In [72]:
inp = np.expand_dims(val_conv_features[0], 0)
np.round(classifier_model_v2.predict(inp)[0],2)

array([ 0.98000002,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.01      ,  0.        ], dtype=float32)

In [73]:
def to_plot(img):
    if K.image_dim_ordering() == 'tf':
        return np.rollaxis(img, 0, 1).astype(np.uint8)
    else:
        return np.rollaxis(img, 0, 3).astype(np.uint8)

In [78]:
input_image.shape

(1, 1, 1, 2048)

In [89]:
# cm = get_cm(inp, 0)

input_image = np.expand_dims(trn_conv_features[0],0)
output_image = conv_fn([input_image])[0]

In [90]:
output_image.shape

(1, 1, 1, 8)

In [29]:
from matplotlib import pyplot as plt
plt.imshow(to_plot(cm), cmap="cool")

AttributeError: 'NoneType' object has no attribute 'ndim'

In [104]:
import matplotlib.pyplot as plt 
# need to get this to show the original image
plt.imshow(to_plot(val[0]))

TypeError: 'DirectoryIterator' object does not support indexing

In [107]:
val = np.concatenate([validation_generator.next() for i in range(validation_generator.n)])

KeyboardInterrupt: 

In [109]:
validation_generator.batch_size/validation_gene

20

In [97]:
resnet_base.layers[-1].output_shape[1:]

(None, None, 2048)

In [94]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

In [95]:
ll = model.layers[-1]


In [98]:
ll.output_shape[1:]

(None, None, 512)

In [99]:
resnet_base.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256                                          
___________________________________________________________________________________________

In [100]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________